In [ ]:
pip install sklearn

In [19]:
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
from sklearn.metrics import accuracy_score

In [2]:
class PostgresConnection(object):
    def __init__(self):
        self.connection = psycopg2.connect(database="ecomdb",
                                           user = "postgres",
                                           password = "postgres",
                                           host = "127.0.0.1",
                                           port = "5432")

    def getConnection(self):
        print("successfully connected to database")
        return self.connection

In [6]:
con = PostgresConnection().getConnection()

successfully connected to database


In [22]:
cur = con.cursor()
st = '''select  t.year,t.month ,i.item_key, sum(ft.total_price) 
               from ecomdb_star_schema.fact_table ft 
               join ecomdb_star_schema.time_dim t on t.time_key = ft.time_key
               join ecomdb_star_schema.item_dim i on i.item_key = ft.item_key
               group by (i.item_key,t.year,t.month)
               order by (i.item_key,t.year,t.month) '''
cur.execute(st)
records = cur.fetchall()
df = pd.DataFrame(list(records), columns=['year', 'month' ,'item_id','total_sales'])
#df.head()
df.dropna()

,year,month,item_id,total_sales
0,2014,1,I00001,1518.0
1,2014,2,I00001,2794.5
2,2014,3,I00001,2921.0
3,2014,4,I00001,3737.5
4,2014,5,I00001,2679.5
...,...,...,...,...
22435,2020,9,I00264,4155.0
22436,2020,10,I00264,6315.0
22437,2020,11,I00264,2910.0
22438,2020,12,I00264,6390.0


In [23]:
df2=df
#df=df.values

In [24]:
df.isnull().sum()

year           0
month          0
item_id        0
total_sales    0
dtype: int64

In [25]:
df.dtypes

year            int64
month           int64
item_id        object
total_sales    object
dtype: object

In [26]:
df3=df2
# checking features
cat = df.select_dtypes(include='O').keys()
# display variabels
cat

Index(['item_id', 'total_sales'], dtype='object')

In [27]:
# unique values in each columns
for x in df.columns:
    #prinfting unique values
    print(x ,':', len(df[x].unique()))

year : 8
month : 12
item_id : 264
total_sales : 4568


In [64]:
data=df
data['year'] = data['year'].astype('category')
data['year'] = data['year'].cat.codes

In [65]:
data

,year,item_name,total_sales
0,0,100% Juice Box Variety 6.75 oz,49365.0
1,1,100% Juice Box Variety 6.75 oz,47160.0
2,2,100% Juice Box Variety 6.75 oz,51135.0
3,3,100% Juice Box Variety 6.75 oz,47295.0
4,4,100% Juice Box Variety 6.75 oz,51045.0
...,...,...,...
2067,3,Zoo Animal Cookies/Crackers,43456.0
2068,4,Zoo Animal Cookies/Crackers,43638.0
2069,5,Zoo Animal Cookies/Crackers,44912.0
2070,6,Zoo Animal Cookies/Crackers,43498.0


In [28]:
newdf=df.iloc[:,:-1]
newdf=pd.get_dummies(newdf,columns = ['item_id','month','year'])

In [29]:
newdf

,item_id_I00001,item_id_I00002,item_id_I00003,item_id_I00004,item_id_I00005,item_id_I00006,item_id_I00007,item_id_I00008,item_id_I00009,item_id_I00010,...,month_11,month_12,year_2014,year_2015,year_2016,year_2017,year_2018,year_2019,year_2020,year_2021
0,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22435,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
22436,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
22437,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
22438,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0


In [31]:
X=newdf
y = df.iloc[:,-1]

In [32]:
from sklearn.model_selection import train_test_split 
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.2,random_state = 10)

In [33]:
# Model initialization
regression_model = LinearRegression()
# Fit the data(train the model)
regression_model.fit(X_train, y_train)
# Predict
y_predicted = regression_model.predict(X_test)

In [35]:
from sklearn.metrics import mean_squared_error, r2_score

In [36]:
# model evaluation
rmse = mean_squared_error(y_test, y_predicted)
r2 = r2_score(y_test, y_predicted)

# printing values
print('Intercept:', regression_model.intercept_)
print('Root mean squared error: ', rmse)

Intercept: 3590206920935329.5
Root mean squared error:  995722.8155637255


In [37]:
regression_model.score(X_test,y_test)

0.8545239983345807

In [42]:
from sklearn import svm
regr = svm.SVR(kernel='linear')
regr.fit(X_train, y_train)

SVR(kernel='linear')

In [43]:
y_predicted = regr.predict(X_test)

In [40]:
# model evaluation
rmse = mean_squared_error(y_test, y_predicted)
r2 = r2_score(y_test, y_predicted)

# printing values
print('Intercept:', regr.intercept_)
print('Root mean squared error: ', rmse)

Intercept: [4233.10608949]
Root mean squared error:  7027165.820972457


In [44]:
regr.score(X_test,y_test)

-0.001938419769585309

In [46]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

regr = make_pipeline(StandardScaler(), svm.SVR(C=1.0, epsilon=0.2))
regr.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svr', SVR(epsilon=0.2))])

In [47]:
y_predicted = regr.predict(X_test)

In [49]:
# model evaluation
rmse = mean_squared_error(y_test, y_predicted)
r2 = r2_score(y_test, y_predicted)

# printing values
#print('Intercept:', regr.intercept_)
print('Root mean squared error: ', rmse)
print('R2: ', r2)

Root mean squared error:  6930446.347002778
R2:  -0.012544463740020051


In [50]:
regr.score(X_test,y_test)

-0.012544463740020051

In [24]:
df=data

In [ ]:
# split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [ ]:
# one hot encode input variables
onehot_encoder = OneHotEncoder(sparse=False)
X = onehot_encoder.fit_transform(X)
# summarize the transformed data
print('Input', X.shape)
print(X)

In [ ]:
# split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [ ]:
print(y_train.shape)

In [ ]:
# Model initialization
regression_model = LinearRegression()
# Fit the data(train the model)
regression_model.fit(X_train, y_train)
# Predict
y_predicted = regression_model.predict(X_test)

In [ ]:
# model evaluation
rmse = mean_squared_error(y_test, y_predicted)
r2 = r2_score(y_test, y_predicted)

# printing values
print('Intercept:', regression_model.intercept_)
print('Root mean squared error: ', rmse)

In [ ]:
plt.scatter(X_train, y_train)

In [ ]:
print(X_train.size)